In [1]:
import numpy as np
import pandas as pd

import arviz as az

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

/home/ec2-user/miniconda/lib/python3.7/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")


Loading BokehJS ...

## Problem 8.2: Microtubule catastrophe, 40 pts

_Note: This problem is best done after the lecture November 22._

In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](../data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. To start with, we will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider three models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.

Note that these descriptions are for the likelihood; we have not specified priors.


**a)**  Describe the three models in words. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Tutorial 3c will be useful. 

<br />



- Model 1: The time to catastrophe is Exponentially distributed.

This suggests that the occurance of catastrophe is a Poisson process, so it is a "rare event" that requires multiple subprocesses to lead it it. The parameter for the process, if it is exponential, $\beta$, represents the characteristic rate of catastrophe, that is how often catastrophe happens in a certain amount of time. It can also be parametrized as $\tau=1/\beta$, the characteristic catastrophe time, which fits what we are given in our data. The Exponential distribution is a special case of the Gamma distribution where $\alpha = 1$ and a special case of the Weibull distribution where $\alpha = 1$ and $\sigma=1/\beta$



- Model 2: The time to catastrophe is Gamma distributed.

This suggests that the occurance of catastrophe represents a specific number of occurances of a Poisson process, that is a discrete number of steps that occur at the same rate must occur for catastrophe to occur. There are two parameters for this distribution, $\alpha$ and $\beta$, where $\alpha$ is the number of arrivals (or "steps") required to trigger catastrophe, and $\beta$ is the rate of the arrivals. Thus, the characteristic catastrophe time is given by $\alpha/\beta$. 


- Model 3: The time to catastrophe is Weibull distributed.

This suggests that the likelihood of catastrophe is dependent on the amount of time it has been since the last catastrophe, so the longer it has been since the last catastrophe, the more likely it is that catastrophe will occur. There are two parameters for this distribution, $\alpha$ which defines how the probability changes over time, and $\sigma$ which is the characteristic catastrophe time.

**b)** Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.

<br />




**c)** Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?


In [2]:
df = pd.read_csv('/home/ec2-user/data/gardner_mt_catastrophe_only_tubulin.csv', comment = "#")

In [3]:
df.tail()

,12 uM,7 uM,9 uM,10 uM,14 uM
687,1335.0,NaN,NaN,NaN,NaN
688,1485.0,NaN,NaN,NaN,NaN
689,1505.0,NaN,NaN,NaN,NaN
690,1520.0,NaN,NaN,NaN,NaN
691,1785.0,NaN,NaN,NaN,NaN


In [5]:
prior_sm1 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m1.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_252dd5c740a0a06b9afaef1a18f767bb NOW.
/home/ec2-user/miniconda/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmppaeqhutm/stanfit4anon_model_252dd5c740a0a06b9afaef1a18f767bb_1618366538660500771.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [7]:
# Store input parameters in a dictionary so stan can access them
data = dict(N=692)

# Generate samples
samples_gen1 = prior_sm1.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)

# Store samples in a dataframe
df_gen1 = bebi103.stan.to_dataframe(samples_gen1, diagnostics=False)
df_gen1.head()

,chain,chain_idx,warmup,uM_12[1],uM_12[2],uM_12[3],uM_12[4],uM_12[5],uM_12[6],uM_12[7],...,uM_12[685],uM_12[686],uM_12[687],uM_12[688],uM_12[689],uM_12[690],uM_12[691],uM_12[692],beta_,lp__
0,1,1,0,7.180542,1.618777,47.512309,4.442333,4.502714,10.214465,33.583216,...,3.386216,11.480054,24.696469,57.743335,16.467838,7.150312,44.742996,24.643234,0.061468,0.0
1,1,2,0,19.459293,8.258657,25.665068,4.189957,15.437581,40.421541,2.062509,...,29.561131,14.385987,20.426809,12.908379,26.964312,28.176392,8.373004,4.534507,0.056857,0.0
2,1,3,0,2.363753,20.865100,20.776067,5.133306,8.251315,0.684464,2.923858,...,88.151229,0.614234,49.865409,8.552279,0.174910,20.563963,37.432141,0.338425,0.052644,0.0
3,1,4,0,39.577807,92.961606,40.626802,104.370375,4.844457,1.020358,31.314712,...,47.295107,47.793295,101.539110,180.095763,91.212056,2.614203,62.111231,53.709735,0.019309,0.0
4,1,5,0,3.047681,18.028469,33.432276,1.400614,47.031483,8.406304,1.622274,...,12.604658,6.928510,22.391221,2.860453,6.884155,12.281826,26.989721,1.910046,0.088738,0.0


In [9]:
p = bebi103.viz.predictive_ecdf(samples_gen1, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 200)
bokeh.io.show(p)

In [ ]:
sm1 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m1.stan')
sm2 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m2.stan')
sm3 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m3.stan')

In [ ]:
data = dict(N=len(df),
           uM_12=df['12 uM'].values.astype(float))



In [ ]:
samples1 = sm1.sampling(data=data)
samples2 = sm2.sampling(data=data)
samples3 = sm2.sampling(data=data)

In [ ]:
df_mcmc1 = bebi103.stan.to_dataframe(samples1, diagnostics=False, inc_warmup=False)

# Take a look
df_mcmc1.head()

In [ ]:
plot = bebi103.viz.ecdf(df_mcmc1["beta_"], x_axis_label="beta_", plot_height=200, plot_width=250) 
bokeh.io.show(plot)

In [ ]:
plot = bebi103.viz.histogram(df_mcmc1["beta_"], x_axis_label="beta_", 
                               plot_height=200, plot_width=250, 
                               bins=30, density=True) 
                 
bokeh.io.show(plot)